In [15]:
%load_ext autoreload
%autoreload 2

from collections import defaultdict

import dvu
import matplotlib.pyplot as plt
import pandas as pd
from os.path import join
import os.path
from tqdm import tqdm
import pathlib
import imodelsx.llm
import json
import requests
import numpy as np
import openai
import pubmed
import paper_parsing
import prompts
openai.api_key = open('/home/chansingh/.OPENAI_KEY').read().strip()
plt.style.use('default')
dvu.set_style()

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


### Load papers and extract text

In [16]:
# pubmed.download_open_source_papers(df) 
df, ids_with_paper = paper_parsing.download_and_check_gsheet()

# extract text from pdfs (create file num.txt for each file num.pdf)
# paper_parsing.extract_texts_from_pdf(ids_with_paper, papers_dir='../papers')

### Extract info from the text (populate the predicted columns)
- num_male, num_female, num_total, num_male_evidence_span, num_female_evidence_span, num_total_evidence_span
- num_white, num_black, num_latino, num_asian, race_evidence_span

In [17]:
# get prompt
llm = imodelsx.llm.get_llm("gpt-4-0613") # gpt-3.5-turbo-0613
imodelsx.llm.LLM_CONFIG['LLM_REPEAT_DELAY'] = 5


# properties, functions, content_str = prompts.get_prompts_gender_and_race()
# print('attempting to add', properties.keys())
# paper_parsing.add_columns_based_on_properties(df, ids_with_paper, properties, functions, content_str, llm)

properties, functions, content_str = prompts.get_prompts_gender()
print('attempting to add', properties.keys())
paper_parsing.add_columns_based_on_properties(df, ids_with_paper, properties, functions, content_str, llm)

properties, functions, content_str = prompts.get_prompts_demographics()
print('attempting to add', properties.keys())
paper_parsing.add_columns_based_on_properties(df, ids_with_paper, properties, functions, content_str, llm)

attempting to add dict_keys(['num_male', 'num_female', 'num_total', 'evidence_span_gender'])


  0%|          | 0/184 [00:00<?, ?it/s]

not cached


### Evaluate

In [13]:
for k in ['num_male', 'num_female']:
    idxs = (df[k + '_corrected'].notnull() & ~(df[k + '_corrected'] == 'Unk'))
    gt = df[k + '_corrected'][idxs].astype(int)
    pred = df[k].apply(paper_parsing.cast_int)[idxs].astype(int)
    acc = (gt == pred).mean()
    print(f'{k} acc={acc:0.2f} n={len(gt)}')

num_male acc=0.81 n=62
num_female acc=0.81 n=62


In [14]:
df.to_csv('../data/main.csv', index=False)